# Day 5: Data Preprocessing

we will compare thousands of apps in the Google Play Store so that we can gain insight into:

How competitive different app categories (e.g., Games, Lifestyle, Weather) are

Which app category offers compelling opportunities based on its popularity

How many downloads you would give up by making your app paid vs. free

How much you can reasonably charge for a paid app

Which paid apps have had the highest revenue

How many paid apps will recoup their development costs based on their sales revenue


**Today you'll learn:**

How to quickly remove duplicates

How to remove unwanted symbols and convert data into a numeric format

How to wrangle columns containing nested data with Pandas

How to create compelling data visualisations with the plotly library

Create vertical, horizontal and grouped bar charts

Create pie and donut charts for categorical data

Use colour scales to make beautiful scatter plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df_apps = pd.read_csv("apps.csv")
df_apps.head()

In [ ]:
print(f"How many rows and columns does df_apps have? \nRows: {df_apps.shape[0]} \nColumns: {df_apps.shape[1]}")

In [ ]:
print(f"Name of columns: {df_apps.columns}")

In [ ]:
print(f"The number of random sample: {df_apps.sample(5)}")

In [ ]:
df_apps.drop(columns=["Last_Updated","Android_Ver"], axis=1, inplace=True)
df_apps.head()

In [ ]:
nan_rows= df_apps[df_apps['Rating'].isna()]
print(nan_rows.shape)
nan_rows.head()

In [ ]:
df_apps_clean = df_apps.dropna()
df_apps_clean.shape

In [ ]:
duplicated_rows = df_apps_clean[df_apps_clean.duplicated()]
print(duplicated_rows.shape)
duplicated_rows.head()

In [ ]:
df_apps_clean[df_apps_clean['App'] == 'Instagram']

In [ ]:
df_apps_clean = df_apps_clean.drop_duplicates()

In [ ]:
df_apps_clean[df_apps_clean['App'] == 'Instagram']

In [ ]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App','Type','Price'])
df_apps_clean[df_apps_clean['App'] == 'Instagram']

In [ ]:
df_apps_clean.shape

In [ ]:
df_apps_clean.sort_values('Rating', ascending=False).head()

In [ ]:
df_apps_clean.sort_values('Size_MBs', ascending=False).head()

In [ ]:
df_apps_clean.sort_values('Reviews', ascending=False).head(50)

In [ ]:
ratings = df_apps_clean.Content_Rating.value_counts()
ratings

In [ ]:
fig = px.pie(labels=ratings.index,
values=ratings.values,
title="Content Rating",
names=ratings.index,
)
fig.update_traces(textposition='outside', textinfo='percent+label')

fig.show()

In [ ]:
fig=px.pie(labels=ratings.index,
           values=ratings.values,
           title="Content Rating",
           names=ratings.index,
           hole=0.6,
          )
fig.update_traces(textposition='inside', textfont_size=15,
                  textinfo='percent')

fig.show()

In [ ]:
df_apps_clean.Installs.describe()

In [ ]:
df_apps_clean.info()

In [ ]:
df_apps_clean[['App','Installs']].groupby('Installs').count()

In [ ]:
df_apps_clean.Installs = df_apps_clean.Installs.astype(str).str.replace(',',"")
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)
df_apps_clean[['App','Installs']].groupby('Installs').count()

Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.



Remove all apps that cost more than $250 from the df_apps_clean DataFrame.



Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest-grossing paid apps according to this estimate? Out of the top 10, how many are games?

In [ ]:
df_apps_clean.Price.describe()

In [ ]:
df_apps_clean.Price = df_apps_clean.Price.astype(str).str.replace('$',"")
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)
df_apps_clean.sort_values('Price', ascending=False).head(20)

In [ ]:
df_apps_clean = df_apps_clean[df_apps_clean['Price'] < 250]
df_apps_clean.sort_values('Price', ascending=False).head(5)

In [ ]:
df_apps_clean['Revenue_Estimate'] = df_apps_clean.Installs.mul(df_apps_clean.Price)
df_apps_clean.sort_values('Revenue_Estimate', ascending=False)[:10]

In [ ]:
df_apps_clean.Category.nunique()

In [ ]:
top10_category = df_apps_clean.Category.value_counts()[:10]
top10_category

In [ ]:
bar = px.bar(x = top10_category.index,
             y = top10_category.values)

bar.show()

In [ ]:
category_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})
category_installs.sort_values('Installs', ascending=True, inplace=True)

In [ ]:
h_bar = px.bar(x = category_installs.Installs,
               y = category_installs.index,
               orientation='h')

h_bar.show()

In [ ]:
h_bar = px.bar(x = category_installs.Installs,
               y = category_installs.index,
               orientation= 'h',
               title = 'Category Popularity')

h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

In [ ]:
cat_number = df_apps_clean.groupby('Category').agg({'App':pd.Series.count})

In [ ]:
cat_merged_df = pd.merge(cat_number, category_installs, on='Category', how='inner')
print(f"The dimensions of the DataFrame are: {cat_merged_df.shape}")
cat_merged_df.sort_values('Installs', ascending=False)

In [ ]:
scatter = px.scatter(cat_merged_df,
                     x='App',
                     y='Installs',
                     title='Category Concentration',
                     size='App',
                     hover_name=cat_merged_df.index,
                     color='Installs')

scatter.update_layout(xaxis_title="Number of Apps (Lower=More Concentrated)",
                      yaxis_title="Installs",
                      yaxis=dict(type='log'))

scatter.show()

Challenge
How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's .stack() method.

In [ ]:
len(df_apps_clean.Genres.unique())

In [ ]:
df_apps_clean.Genres.value_counts().sort_values(ascending=True)[:5]

In [ ]:
stack = df_apps_clean.Genres.str.split(';', expand=True).stack()
print(f"We now have a single column with shape: {stack.shape}")
num_genres = stack.value_counts()
print(f"Number of genres: {len(num_genres)}")

In [ ]:
bar = px.bar(x = num_genres.index[:15],
             y = num_genres.values[:15],
             title='Top Genres',
             hover_name = num_genres.index[:15],
             color=num_genres.values[:15],
             color_continuous_scale = 'Agsunset')

bar.update_layout(xaxis_title='Genre',
                  yaxis_title='Number of Apps',
                  coloraxis_showscale=False)

bar.show()

In [ ]:
df_apps_clean.Type.value_counts()

In [ ]:
df_free_vs_paid = df_apps_clean.groupby(["Category", "Type"],
                                        as_index=False).agg({'App':pd.Series.count})
df_free_vs_paid.head()

In [ ]:
g_bar = px.bar(df_free_vs_paid,
               x='Category',
               y='App',
               title='Free vs Paid Apps by Category',
               color='Type',
               barmode='group')

g_bar.update_layout(xaxis_title='Category',
                    yaxis_title='Number of Apps',
                    xaxis={'categoryorder':'total descending'},
                    yaxis=dict(type='log'))

g_bar.show()

In [ ]:
box = px.box(df_apps_clean,
             y='Installs',
             x='Type',
             color='Type',
             notched=True,
             points='all',
             title='How Many Downloads are Paid Apps Giving Up?')

box.update_layout(yaxis=dict(type='log'))

box.show()

In [ ]:
df_paid_apps = df_apps_clean[df_apps_clean['Type'] == 'Paid']
box = px.box(df_paid_apps,
             x='Category',
             y='Revenue_Estimate',
             title='How Much Can Paid Apps Earn?')

box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Ballpark Revenue',
                  xaxis={'categoryorder':'min ascending'},
                  yaxis=dict(type='log'))

box.show()

In [ ]:
box = px.box(df_paid_apps,
             x='Category',
             y='Price',
             title='Price per Category')

box.update_layout(xaxis_title='Category',
                  yaxis_title='Paid App Price',
                  xaxis={'categoryorder':'max descending'},
                  yaxis=dict(type='log'))
box.show()